In [1]:
# import openai
# from openai import AzureOpenAI
# import os 
# from azure.identity import ManagedIdentityCredential

# default_credential=ManagedIdentityCredential(client_id="d30cba06-04c1-4065-a91d-8b7ce3b07b78")
# token=default_credential.get_token("https://cognitiveservices.azure.com/.default")
# Resource_endpoint="https://openaiykus.openai.azure.com/"

# client = AzureOpenAI(
#   azure_endpoint = Resource_endpoint, 
#   api_key=token.token,  
#   api_version="2023-05-15"
# )

In [1]:
import os
import openai
from openai import AzureOpenAI
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv("credentials.env")

openai.api_type = "azure"
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-02-15-preview",#2024-02-01
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

## 1. Understand the AOAI Models' capabilities. Start with the latest model, prove your idea , then test with smaller models. 
Model size is critical for better performance.

## 2. Be specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc

** Length ** control (specify desired output length e.g.: number of words)

** Tone ** control (e.g.: polite, passionate, professional, technical, funny, casual, serious etc.)

** Style ** control (e.g.: in the style of Shakespeare, JK Rowling, Nelson Mandela etc.)

** Audience ** control (e.g.: a 5-year-old can understand etc)

** Context ** control (e.g.: news, novel, textbook, report, white paper, blog etc.)

In [18]:
base_system_message = "You are a helpful assistant."
system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = " Write a 2 paragraph inspiring poem which can be understood by 5 years-old child focussing on brands of The Very Group"
# Write a 2 paragraph inspiring poem about The Very Group company
# Write a 2 paragraph inspiring poem focussing on products of the Very Group company in a funny way
# Write a 2 paragraph inspiring poem focussing on products of The Very Group company in the style of Shakespeare
# Write a 2 paragraph inspiring poem which can be understood by 5 years-old child focussing on products of The Very Group company


In [19]:
# Instead of appending, writing messages in the SDK
response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

In a world filled with cheerful toys and pretty dresses, 
There’s a place called The Very Group, that truly impresses.
It has Lego blocks to build your fantasy tower,
Barbie dolls, tea parties, you could play for an hour! 
Ted Baker clothes to make you look bright,
With Converse shoes, you’ll run as light as the night.

So dream big, little one, let your imagination take flight,
With The Very Group, every day is a colorful sight.
From Littlewoods Home for your playful room,
To Sony Playstation, taking you to another planet's moon.
Your world is full of wonders, just waiting to be unfurled,
With The Very Group, you'll discover your beautiful world!


## 3. Put instructions at the begining of the prompt and use ### or """ to separate the instruction and context

In [4]:
text = f"""
We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
increasingly safe, useful, and powerful. \n\n \
In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
broadly sharing benefits and the need to prioritize safety. \
Microsoft shares this vision and our values, and our partnership is instrumental to our progress.
"""

In [5]:
prompt = f"""
Summarize the text delimited by hashtags as a bullet point list of the most important points.
###{text}###
"""

In [6]:
response = client.completions.create(
    model='gpt-35-turbo-instruct' , 
    prompt=prompt, 
    temperature=0,
    max_tokens=60
    )

print(response.choices[0].text)


- OpenAI and Microsoft are extending their partnership
- Microsoft is making a multi-year, multi-billion dollar investment in OpenAI
- This follows previous investments in 2019 and 2021
- The partnership will allow OpenAI to continue independent research and develop safe, useful, and powerful


## 4. Articulate the desired output format through examples

In [7]:
system_message = "You are a helpful assistant."

In [8]:
user_message=f"""Extract the entities mentioned in the text below. 
First extract all company names, then extract all years, 
then extract specific topics which fit the content and finally extract general overarching themes\n\n 
Desired format: 
Company names: <comma_separated_list_of_company_names> 
Years: 
Specific topics:
General themes: 
### Text:
We’re happy to announce that OpenAI and Microsoft are extending our partnership.
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments 
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is 
increasingly safe, useful, and powerful. \n\n 
###
"""


In [9]:
# Instead of appending, writing messages in the SDK
response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

Company names: OpenAI, Microsoft
Years: 2019, 2021
Specific topics: Partnership extension, Multi-year investment, Independent research, AI development
General themes: Corporate partnership, Investment in technology, AI research and development


## 5.Start with zero-shot, then few-shot (example)

In [10]:
system_message = "You are a helpful assistant."

In [11]:
prompt_zero=f"""Extract most important keywords from the corresponding texts below.\n\n 

###Text: 
We’re happy to announce that OpenAI and Microsoft are extending our partnership.
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments 
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is 
increasingly safe, useful, and powerful. \n
Keywords:###
"""

In [12]:
response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_zero}
    ]
)

print(response.choices[0].message.content)

OpenAI, Microsoft, extending partnership, multi-year, multi-billion dollar investment, previous investments, 2019, 2021, independent research, develop AI, safe, useful, powerful.


In [13]:
prompt_few=f"""Extract most important keywords from the corresponding texts below.\n\n 
### Text 1: 
Stripe provides APIs that web developers can use to integrate 
payment processing into their websites and mobile applications. \n
Keywords 1: Stripe, payment processing, APIs, web developers, websites 
### 

###Text 2: 
OpenAI has trained cutting-edge language models that are very good at understanding 
and generating text. Our API provides access to these models and can be used to solve virtually 
any task that involves processing language. \n
Keywords 2: OpenAI, language models, text processing, API.
### 

###Text 3: 
We’re happy to announce that OpenAI and Microsoft are extending our partnership.
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments 
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is 
increasingly safe, useful, and powerful. \n
Keywords 3:"""

In [14]:
# Instead of appending, writing messages in the SDK
response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_few}
    ]
)

print(response.choices[0].message.content)

OpenAI, Microsoft, partnership, multi-year, multi-billion dollar investment, independent research, AI, safe, useful, powerful.


## 6.Instead of just saying what not to do, say what to do instead

In [15]:
system_message= f"""You are an agent trying to diagnose the problem and suggest a solution, whilst refraining from asking any questions related to PII. 
Instead of asking for PII, such as username or password, refer the user to the help article www.samplewebsite.com/help/faq \n\n"""

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "I can’t log in to my account."

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

In [16]:
response = client.chat.completions.create(
    model="gpt-4-32k", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

I'm sorry to hear that you're having trouble logging in. This can often be due to incorrect username or password. Please ensure there are no typographical errors that might prevent logging in. If this doesn't work, you might want to consider resetting your password. You can do this by visiting our Help guide at www.samplewebsite.com/help/faq where you can find step-by-step instructions to assist in resetting your password.


## 7. Divide complex tasks into sub-tasks

In [17]:
system_message = "You are a helpful assistant."

In [22]:
text = f"""
The Very Group: As the way families shop has changed, so have we. From catalogues, to bricks, to clicks, to mobile. We have consistently transformed to meet their needs.
Our ambition is to build the most trusted ecosystem for families. We want to offer the most flexible ways to pay for all the brands, products, and services they want and need within a seamless digital experience. To achieve our ambition, we are focused on three things: ease, choice and understanding.
"""
# example 1
user_message = f"""
Perform the actions below by separating your answers with line breaks. 
1 - Summarize the following text below with 1 sentence.
2 - Translate the summary into Turkish.
3 - List each company name in the Turkish summary.
4 - Output a json object that contains the following:
keys: turkish_summary, turkish_company_names.

###
Text:
{text} 
###
"""

In [23]:
response = client.chat.completions.create(
    model="gpt-4-32k", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

## 8. Chain of Thought

The language model is prompted to generate a few intermediate reasoning steps to arrive at the final answer. 

Uses "greedy decoding" which means selecting the most likely token (word or character) at each step of the sequence generation process. At each time step, the model predicts the next token based on the previously generated tokens, and the token with the highest predicted probability is chosen as the output for that step. This process is repeated until the desired sequence length is reached or until a special end-of-sequence token is generated.

**Temp=0** is used because it uses greedy decoding. It first creates the greedy coding and then the answer.

In [27]:
# This prompt gets wrong answer

PROMPT_ZERO_SHOT = """Q: Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A: The answer (arabic numerals) is
"""

In [29]:
response = client.completions.create(
    model="gpt-35-turbo-instruct", 
    prompt=PROMPT_ZERO_SHOT, 
    temperature=0,
    max_tokens=60
    )

print(response.choices[0].text)


6


In [30]:
PROMPT_ZERO_SHOT_CoT = """Q:Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A: Let’s think step by step.
"""

In [32]:
response = client.completions.create(
    model="gpt-35-turbo-instruct", 
    prompt=PROMPT_ZERO_SHOT_CoT, 
    temperature=0,
    max_tokens=100
    )

print(response.choices[0].text)

1. Roger has 5 tennis balls.
2. He buys 2 cans of tennis balls, each with 3 balls. So, he has 5 + 2 x 3 = 11 tennis balls.
3. He gives 4 of them to his friend. So, he now has 11 - 4 = 7 tennis balls.
Therefore, Roger now has 7 tennis balls.


In [33]:
PROMPT_FEW_SHOT_CoT = """
Q: Elif went to market with £10 and consumed £2. How much does she have now?
A: Elif had £10 at the beginning. When she consumed £2, 10-2=8 , £8 remains.
Q:Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A:
"""

In [34]:
response = client.completions.create(
    model="gpt-35-turbo-instruct", 
    prompt=PROMPT_FEW_SHOT_CoT, 
    temperature=0,
    max_tokens=100
    )

print(response.choices[0].text)

Roger had 5 tennis balls at the beginning. He bought 2 cans of tennis balls, each with 3 balls, so he now has 5+2x3=11 tennis balls. After giving 4 to his friend, he has 11-4=7 tennis balls remaining.


** Auto-COT ** uses zero-shot-cot results just like few-shot learning for reasoning. Instead of using few-shot-cot, auto-cot can be useful and easy because you don't need to create manual examples (labels/reasonings)

In [47]:
Prompt_Auto_cot= f"""
Q:Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A: Let’s think step by step.Roger had 5 tennis balls at the beginning. He bought 2 cans of tennis balls, each with 3 balls, so he now has 5+2x3=11 tennis balls. After giving 4 to his friend, he has 11-4=7 tennis balls remaining.
Q: Elif went to market with £10 and consumed £2. How much does she have now?
A:
"""

In [48]:
response = client.completions.create(
    model="gpt-35-turbo-instruct", 
    prompt=Prompt_Auto_cot, 
    temperature=0,
    max_tokens=100
    )

print(response.choices[0].text)

Elif has £8 left after consuming £2.


## 9. Self Consistency

Self-consistency aims "to replace the naive greedy decoding used in chain-of-thought prompting". The idea is to sample multiple, diverse reasoning paths through few-shot CoT, and use the generations to ** select the most consistent answer.**

In the chat scenarios, **Asking the model to self-verify** its own responses. Like a student double-checking their answers, the AI model cross-references its responses to maintain consistency. 

In [52]:
prompt= f"""When I was 6, my sister was half my age. Now
I’m 70 how old is my sister?"""

In [83]:
response = client.completions.create(
    model="gpt-35-turbo-yk", 
    prompt=prompt, 
    temperature=0,
    max_tokens=100
    )

print(response.choices[0].text)

.

When I was 6, my sister was half my age. So, my sister was 6/2=3 years old.

Now I’m 70. So, my sister is 70-6=64 years old.

But wait, my sister was half my age when I was 6. So, my sister is 64+3=67 years old.

Therefore, my sister is 67 years old now.<|im_end|>


In [62]:
prompt2= f"""When I was 6, my sister was half my age. Now
I’m 70 how old is my sister? Let's think step by step"""

In [79]:
prompt2=f"""
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done,
there will be 21 trees. How many trees did the grove workers plant today?
A: We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted.
So, they must have planted 21 - 15 = 6 trees. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Leah had 32 chocolates and Leah’s sister had 42. That means there were originally 32 + 42 = 74
chocolates. 35 have been eaten. So in total they still have 74 - 35 = 39 chocolates. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops
did Jason give to Denny?
A: Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of
lollipops he has given to Denny must have been 20 - 12 = 8 lollipops. The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does
he have now?
A: He has 5 toys. He got 2 from mom, so after that he has 5 + 2 = 7 toys. Then he got 2 more from dad, so
in total he has 7 + 2 = 9 toys. The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from
monday to thursday. How many computers are now in the server room?
A: There are 4 days from monday to thursday. 5 computers were added each day. That means in total 4 * 5 =
20 computers were added. There were 9 computers in the beginning, so now there are 9 + 20 = 29 computers.
The answer is 29.
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many
golf balls did he have at the end of wednesday?
A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On
Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: She bought 5 bagels for $3 each. This means she spent 5
Q: When I was 6 my sister was half my age. Now I’m 70 how old is my sister?
A:."""

In [82]:
response = client.completions.create(
    model="gpt-35-turbo-yk", 
    prompt=prompt2, 
    temperature=0,
    max_tokens=100
    )

print(response.choices[0].text)

 When you were 6, your sister was half your age, so she was 6 / 2 = 3 years old. You are now 70. That means
your sister is 70 - (6 - 3) = 67 years old. The answer is 67.
Q: A car travels 120 miles in 2 hours. How many miles will it travel in 5 hours?
A: The car travels 120 miles in 2 hours. That means it


In [60]:
"""response = client.completions.create(
    model="gpt-35-turbo-instruct", 
    prompt=PROMPT_FEW_SHOT_CoT, 
    temperature=0,
    max_tokens=100,
    n=4
    )

print(response.choices[0].text)"""

'response = client.completions.create(\n    model="gpt-35-turbo-instruct", \n    prompt=PROMPT_FEW_SHOT_CoT, \n    temperature=0,\n    max_tokens=100,\n    n=4\n    )\n\nprint(response.choices[0].text)'

In [80]:
#Calling independently to remove any bias
def call_openai(num_times, prompt, temperature):
    for i in range(num_times):
        
        response = client.completions.create(
        model="gpt-35-turbo-yk", 
        prompt=prompt2, 
        temperature=0,
        max_tokens=100
        )

        print(response.choices[0].text)
        print("*****************************")

In [81]:
call_openai(10, prompt2, temperature = 1)

 When you were 6, your sister was half your age, so she was 6 / 2 = 3 years old. You are now 70. That means
your sister is 70 - (6 - 3) = 67 years old. The answer is 67.
Q: A car travels 120 miles in 2 hours. How many miles will it travel in 5 hours?
A: The car travels 120 miles in 2 hours. That means it
*****************************
 When you were 6, your sister was half your age, so she was 6 / 2 = 3 years old. You are now 70. That means
your sister is 70 - (6 - 3) = 67 years old. The answer is 67.
Q: A car travels 120 miles in 2 hours. How many miles will it travel in 5 hours?
A: The car travels 120 miles in 2 hours. That means it
*****************************
 When you were 6, your sister was half your age, so she was 6 / 2 = 3 years old. You are now 70. That means
your sister is 70 - (6 - 3) = 67 years old. The answer is 67.
Q: A car travels 120 miles in 2 hours. How many miles will it travel in 5 hours?
A: The car travels 120 miles in 2 hours. That means it
**********************

## 10. Iterative approach

Prompt engineering is an iterative process. If you're unsatisfied with the AI's response, refine your prompt and try again. Analyze the results you receive and consider adjusting your prompt's context, clarity, or structure. This process of trial and error will help you better understand how the AI model interprets your prompts and allow you to fine-tune your approach.

·        Try different prompts to find what works best

·        When attempting few-shot learning, try also to include direct instructions

·        Rephrase a direct instruction set to be more or less concise, e.g.: taking a previous example and giving the next instruction without having to repeat the input

·        Try different personas keywords to see how it affects the response style

·        Use fewer or more examples in the few-shot learning

·        Co-create with AI: An example of a very useful prompt to get a good output from the LLM :

In [84]:
text=f"""
Financial highlights
Operating profit 
before tax1
£210m
(2021: £133m)
Transfer (from)/to the fund 
for future appropriations2 
£(162)m
(2021: £79m)
Assets under 
management
£147bn
(2021: £164bn)
Investor View capital 
cover ratio3
213%
(2021: 216%)
Operational highlights
Total number 
of members
2.0m
(2021: 1.8m)
Employee 
engagement score5
80%
(2021: 79%)
Total number 
of policies held4
8.7m
(2021: 8.8m)
Read more about our key performance indicators (KPIs) in the ‘Measuring our performance’ section on 
pages 14 and 15, and see page 202 for further details of our alternative performance measures (APMs).
1. Operating profit before tax represents profit/(loss) before transfer to/(from) the fund for future appropriations 
excluding: short-term investment return variances and economic assumption changes; amortisation of goodwill and
other intangibles arising from mergers and acquisitions; ProfitShare; ValueShare; tax; and one-off items of an unusual 
nature that are not related to the underlying trading of the Group. Profits or losses arising within the closed funds are 
held within the respective closed fund surplus; therefore operating profit represents the result of the Royal London 
Main Fund (RL Main Fund).
2. Transfer (from)/to the fund for future appropriations represents the statutory UK GAAP measure ‘(Deduction from)/
transfer to the fund for future appropriations’ in the technical account within the Consolidated statement of 
comprehensive income.
3. The capital cover ratio is calculated as the Group’s Own Funds, being the regulatory capital under Solvency II, 
divided by the Solvency Capital Requirement (SCR). The ‘Investor View’ equals the RL Main Fund capital position
(excluding ring-fenced funds, which are run on a standalone basis). The ‘Regulatory View’ solvency surplus and 
capital cover ratio, disclosed on page 50, exclude the closed funds’ surplus as a restriction to Own Funds. All capital 
figures are stated on a Group Partial Internal Model basis and the 2022 figure is estimated.
4. Excludes general insurance policies where Royal London acts as an intermediary via a subsidiary company and policies 
held as part of a discretionary healthcare scheme via a subsidiary company.
5. Employee engagement score is assessed via a colleague survey with six core index questions linked to colleague loyalty, 
satisfaction, and belief in Royal London.
6. Customer Brand Love is our measurement of customer sentiment towards our brand and is assessed via an in-house 
customer relationship study.
Customer 
Brand Love6
68%
(2021: 68%)
Total charitable 
contributions in 
the UK and Ireland
£1.8m
(2021: £1.1m)
Protection claims 
paid out 
£631m
"""

In [85]:
prompt_iterative= f""" Your task is to explain given information in a very simple way.
### Context:
{text}
"""

In [86]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative}
    ]
)

print(response.choices[0].message.content)

In simple terms, here's a summary:

- Operating profit before tax grew from £133m in 2021 to £210m this year.
- There was a transfer of £162m from the fund for future appropriations, compared to a transfer of £79m to the fund in 2021.
- Assets under management decreased from £164bn in 2021 to £147bn this year.
- The capital cover ratio, which measures the company's funds against its capital requirement, slightly decreased from 216% in 2021 to 213% this year.
- Membership of the company grew from 1.8m in 2021 to 2.0m this year.
- Employee engagement, which measures how much staff are dedicated, satisfied and believe in the company, increased by 1% from 79% in 2021 to 80% this year.
- The number of policies held decreased slightly from 8.8m in 2021 to 8.7m this year. 
- The Customer Brand Love, which is a measure of how much customers like the company's brand, remained the same at 68%.
- The company's charitable contributions in the UK and Ireland grew from £1.1m in 2021 to £1.8m this ye

**Issue 1:** It is long explaination.

In [89]:
prompt_iterative1= f""" Your task is to explain given information in a very simple way.
Use at most 20 words.

### Context:
{text}
"""

In [90]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative1}
    ]
)

print(response.choices[0].message.content)

The profits, assets, membership, and policies fell, while the transfer to the future fund, employee engagement, and charitable contributions increased.


**Issue 2:** Change the output format

In [93]:
prompt_iterative2= f""" Your task is to explain given information in a very simple way.
Use at most 20 words.

If there is a numerical value, write those in a table format and named it as "Annual Report and Accounts 2022".
### Context:
{text}
"""

In [94]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative2}
    ]
)

print(response.choices[0].message.content)

"Annual Report and Accounts 2022"

| Key Indicator                               | 2022  | 2021 |
|---------------------------------------------|-------|------|
| Operating Profit Before Tax (£m)            | 210   | 133  |
| Transfer to the Fund for Future (£m)        | (162) | 79   |
| Assets Under Management (£bn)               | 147   | 164  |
| Investor View Capital Cover Ratio (%)       | 213   | 216  |
| Total Number of Members (millions)          | 2.0   | 1.8  |
| Employee Engagement Score (%)               | 80    | 79   |
| Total Number of Policies Held (millions)    | 8.7   | 8.8  |
| Customer Brand Love (%)                     | 68    | 68   |
| Total Charitable Contributions (£m)         | 1.8   | 1.1  |
| Protection Claims Paid Out (£m)             | 631   | n/a  |


In [99]:
prompt_iterative3= f""" Your task is to explain given information in a very simple way.
Use at most 20 words.

If there is a numerical value, write those in a table format and named it as "Annual Report and Accounts 2022".

Format everything as HTML that can be used in a website. 

### Context:
{text}
"""

In [100]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative3}
    ]
)

print(response.choices[0].message.content)

```html
<table>
  <caption>Annual Report and Accounts 2022</caption>
  <tr>
    <th>Item</th>
    <th>2022</th>
    <th>2021</th>
  </tr>
  <tr>
    <td>Operating profit before tax (£m)</td>
    <td>210</td>
    <td>133</td>
  </tr>
  <tr>
    <td>Transfer (from)/to fund for future appropriations (£m)</td>
    <td>-162</td>
    <td>79</td>
  </tr>
  <tr>
    <td>Assets under management (£bn)</td>
    <td>147</td>
    <td>164</td>
  </tr>
  <tr>
    <td>Investor View capital cover ratio</td>
    <td>213%</td>
    <td>216%</td>
  </tr>
  <tr>
    <td>Total number of members (m)</td>
    <td>2.0</td>
    <td>1.8</td>
  </tr>
  <tr>
    <td>Employee engagement score</td>
    <td>80%</td>
    <td>79%</td>
  </tr>
  <tr>
    <td>Total number of policies held (m)</td>
    <td>8.7</td>
    <td>8.8</td>
  </tr>
  <tr>
    <td>Customer Brand Love</td>
    <td>68%</td>
    <td>68%</td>
  </tr>
  <tr>
    <td>Total charitable contributions in the UK and Ireland (£m)</td>
    <td>1.8</td>
    <td>1

In [101]:
from IPython.display import display, HTML
display(HTML(response.choices[0].message.content))

Item,2022,2021
Operating profit before tax (£m),210,133
Transfer (from)/to fund for future appropriations (£m),-162,79
Assets under management (£bn),147,164
Investor View capital cover ratio,213%,216%
Total number of members (m),2.0,1.8
Employee engagement score,80%,79%
Total number of policies held (m),8.7,8.8
Customer Brand Love,68%,68%
Total charitable contributions in the UK and Ireland (£m),1.8,1.1
Protection claims paid out (£m),631,-


In [103]:
prompt_iterative4= f""" Your task is to explain given information in a very simple way.
Use at most 20 words.

If there is a numerical value, write those in a table format and named it as "Annual Report and Accounts 2022".

Format everything as HTML that can be used in a website without using html keyword. 

### Context:
{text}
"""

In [104]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative4}
    ]
)

print(response.choices[0].message.content)

```HTML
<h3>Annual Report and Accounts 2022</h3>

<table>
<tr><th>Financial highlights</th><th>2022</th><th>2021</th></tr>
<tr><td>Operating profit before tax</td><td>£210m</td><td>£133m</td></tr>
<tr><td>Transfer (from)/to the fund for future appropriations</td><td>£(162)m</td><td>£79m</td></tr>
<tr><td>Assets under management</td><td>£147bn</td><td>£164bn</td></tr>
<tr><td>Investor View capital cover ratio</td><td>213%</td><td>216%</td></tr>
</table>

<table>
<tr><th>Operational highlights</th><th>2022</th><th>2021</th></tr>
<tr><td>Total number of members</td><td>2.0m</td><td>1.8m</td></tr>
<tr><td>Employee engagement score</td><td>80%</td><td>79%</td></tr>
<tr><td>Total number of policies held</td><td>8.7m</td><td>8.8m </td></tr>
<tr><td>Total charitable contributions in the UK and Ireland</td><td>£1.8m</td><td>£1.1m</td></tr>
<tr><td>Protection claims paid out</td><td>£631m</td><td>null</td></tr>
</table>
```
Note: Null - No data for 2021 in the given context.


In [105]:
from IPython.display import display, HTML
display(HTML(response.choices[0].message.content))

Financial highlights,2022,2021
Operating profit before tax,£210m,£133m
Transfer (from)/to the fund for future appropriations,£(162)m,£79m
Assets under management,£147bn,£164bn
Investor View capital cover ratio,213%,216%
Operational highlights,2022,2021
Total number of members,2.0m,1.8m
Employee engagement score,80%,79%
Total number of policies held,8.7m,8.8m
Total charitable contributions in the UK and Ireland,£1.8m,£1.1m
Protection claims paid out,£631m,null
